In [90]:
from sqlite3 import connect
import sys
from os import curdir, path

In [91]:
modules_dir = path.join(curdir, '..', 'modules')
sys.path.append(modules_dir)

In [92]:
from db import read_pages

In [93]:
db = connect(path.join(curdir, '..', 'articles.sqlite'))

In [94]:
pages = db.execute("SELECT * FROM {}".format('articles')).fetchall()

In [95]:
pages[0]


('https://vc.ru/marketing/63026-12-sovetov-kak-zarabotat-bolshe-podpischikov-v-instagram',
 '12 советов, как заработать больше подписчиков в Instagram — Маркетинг на vc.ru',
 '1. Продумайте контент-стратегию!')

In [96]:
titles = list(map(lambda x: x[1], pages))
descriptions = list(map(lambda x: x[2], pages))

In [97]:
print(titles[0])
print(descriptions[0])

12 советов, как заработать больше подписчиков в Instagram — Маркетинг на vc.ru
1. Продумайте контент-стратегию!


In [98]:
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import csc_matrix

In [105]:
cv = CountVectorizer()

In [106]:
data_matrix = csc_matrix(cv.fit_transform(titles + descriptions))

In [107]:
data_matrix

<240x1283 sparse matrix of type '<class 'numpy.int64'>'
	with 4149 stored elements in Compressed Sparse Column format>

In [115]:
tokens = cv.get_feature_names()

In [111]:
import numpy as np

In [146]:
eps = 10e-8

In [147]:
class Embedding:
    def __init__(self, matrix, epsilon=eps):
        self.data = matrix
        self.sigma = np.zeros(matrix.shape)
        self.sigma.fill(epsilon)
    @staticmethod
    def create(emb_size, vocab_size, rng = 0.001):
        rnd = np.random.randn(emb_size, vocab_size) * rng
        return Embedding(rnd)


In [148]:
emb = Embedding.create(256, len(tokens))

In [149]:
print(emb.sigma.shape)
print(emb.data.shape)

(256, 1283)
(256, 1283)


In [150]:
def backward_hinge(u, v, v_hat, gamma):
    loss = gamma - np.dot(u, v) + np.dot(u, v_hat)
    if loss > 0:
        return (v_hat - v, -u, u)

In [195]:
def update(embedding, idx, max_indicies, delta, nu):
    p = emb.data[:, idx][:, max_indicies]
    print(p)
    g = emp.sigma[:, idx][:, max_indicies]
    emb.sigma[:, idx][:, max_indicies] += delta**2
    emp.data[:, idx][:, max_indicies] -= delta * nu / (np.sqrt(g) + eps)

In [194]:
def aggregate(matrix, idx):
    res = matrix[:, idx]
    maximum = np.max(res, axis=1)
    max_indicies = np.argmax(res, axis=1)
    return maximum, max_indicies

In [ ]:
def recall_at_k(emb, data, k = 10):
    n_test = len(data)
    
    descr_emb = [emb(data[i][2]) for i in range(n_test)]
    
    # я так понял, что нужно посчитать сумму bool(i in top_k)
#     recall = count(1:n_test) do i
#         title_emb = emb(data[i][1])
#         i in top_k(vec(title_emb' * descr_emb), k)
#     end
    
    return recall / n_test

In [ ]:
def train2(emb, idx, nu):
        

In [ ]:
def train1(emb, data, nu):
    first = np.random.permutation(len(data))
    second = np.roll(first, 1)
    for (f, s) in zip(first, second):
        u, v = data[f]
        np.random.randint(2) ? np.setdiff1d(v, u) : np.setdiff1d(u, v)
        train2(emb, (u, v, data[s][2]), nu)

In [ ]:
def train(emb, train_data, test_data, n_epochs, nu):
    for epoch in range(n_epochs):
        t = train1(emb, train_data, nu)
        recall = recall_at_k(emb, test_data)
        print("Epoch {} ({}): recall = {}\n".format(epoch, t, recall))

0
